In [1]:
import cv2
import numpy as np

In [2]:
path="1.jpg"

In [3]:
def get_limits(color):
    c=np.uint8([[color]]) #here insert the bgr value
    hsvB=cv2.cvtColor(c,cv2.COLOR_BGR2HSV)
    lowerLimit = hsvB[0][0][0] - 10, 100, 100
    upperLimit = hsvB[0][0][0] + 10, 255, 255

    lowerLimit=np.array(lowerLimit,dtype=np.uint8)
    upperLimit=np.array(upperLimit,dtype=np.uint8)

    return lowerLimit,upperLimit

In [4]:
blue=[255,0,0]

In [5]:
img=cv2.imread(path)
hsv_blue=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

lowerLimit,upperLimit=get_limits(color=blue)

mask=cv2.inRange(hsv_blue,lowerLimit,upperLimit)

def detect_shape(contour):
    # Approximate the contour to determine the shape
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * peri, True)
    num_sides = len(approx)

    # Circle detection
    if num_sides >= 8:
        area = cv2.contourArea(contour) #area
        radius = np.sqrt(area / np.pi) #radious of circle
        c = peri / (2 * np.pi * radius) #peri
        if c >= 0.8 and c <= 1.2:  #range of circle
            return "Circle"
    return 'Unknown'

val,thresh=cv2.threshold(mask,115,255,cv2.THRESH_BINARY)

contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Iterate over the contours and detect shapes
for contour in contours:
    shape = detect_shape(contour)
    M = cv2.moments(contour)
    cx = int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
    cy = int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0
    cv2.putText(img, shape, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128,128,178), 2)

cv2.imshow('img',mask)
cv2.imshow('img1',img)
cv2.imshow('img2',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()